## Delay Report
### Overview
The delay report script aims to find the updated_eta and updated_etd of certain vessels provided within "Vessel Delay Tracking.XLSX". This is done by querying an underlying BigSchedules API, MSC Web API and from a static G2 Schedules Excel document. None of the API interactions use the Rio Tinto credentials to ensure that traceback cannot occur.

The script is written in a modular approach to increase ease of maintenance and improve code quality. Configurations are stored in a `data` subdirectory. The script expects a `Vessel Delay Tracking.XLSX` file and `g2_filename` (G2 Schedule Excel file) in the same directory.

In [2]:
# Imports
import pandas as pd
import numpy as np
import random
import logging
import os
import json
import requests

from pathlib import Path
from datetime import datetime

In [3]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [4]:
# Read configuration file
with open("data/config.json", "r") as f:
    config = json.load(f)
    
# Used to map carrier names to the ones BigSchedule uses and supports
with open("data/carrier_mapping.json", "r") as f:
    carrier_mapping = json.load(f)

# Bigschedule login
with open("data/bigschedules_login.json", "r") as f:
    bs_login = json.load(f)
    
# Prepare base information
# UNLOCODE to port name mapping
port_mapping = (
    pd.concat([pd.read_csv(p, usecols=[1, 2, 4, 5], engine='python', names=[
              'country', 'port', 'name', 'subdiv']) for p in Path('data').glob("*UNLOCODE CodeListPart*")])
    .query('port == port')
    .assign(
        uncode=lambda x: x.country.str.cat(x.port),
        full_name=lambda x: np.where(
            x.subdiv.notnull(), x.name.str.cat(x.subdiv, sep=", "), x.name)
    )
    .drop_duplicates('uncode')
    .set_index('uncode')
    .to_dict('index')
)

# Read the vessel delay tracking file
xl = pd.ExcelFile('Vessel Delay Tracking.xlsx')

### BSExtractor

In [ ]:
bigschedules_sheet = (
    xl.parse(pd.to_datetime(xl.sheet_names,
                            errors='coerce',
                            format='%d.%m.%Y').max().date().strftime('%d.%m.%Y'),
                            parse_dates=True)
                            .query(f"`Fwd Agent` in {[k for k,v in carrier_mapping.items() if v != '']}")
                            .replace({'Fwd Agent': carrier_mapping})
)

In [ ]:
# Get port name
bigschedules_sheet = bigschedules_sheet.assign(
    pol_name=lambda x: x['Port of Loading'].apply(
        lambda y: port_mapping.get(y)['name']),
    pod_name=lambda x: x['Port of discharge'].apply(
        lambda y: port_mapping.get(y)['name']),
)

In [ ]:
# Determine what searches need to be made (splitting of concerns amongst BigSchedules, MSC & G2)

In [ ]:
# Make the searches on the BigSchedules portal
'''
Takes in a list of dataframe of vessels, their carriers, POL, POD.
Shrinks the above dataframe to vessels & their carriers. Uses this new dataframe for querying BigSchedules.
Outputs an updated dataframe of vessels & their carriers with 2 additional columns updated_eta and updated_etd.
'''

### MSCExtractor

In [5]:
# Get the MSC delay sheet
msc_delay_sheet = (xl.parse(pd.to_datetime(xl.sheet_names,
                            errors='coerce',
                            format='%d.%m.%Y').max().date().strftime('%d.%m.%Y'),
                            parse_dates=True)
                            .query(f"`Fwd Agent` in {['MSC']}"))
)

In [42]:
# Get the MSC-specific port names from the UNLOCODEs
msc_port_mapping = {v['Port Code']: v['MSC Port Name'] for k,v in (pd.read_excel('data/MSC Port Code Mapping.xlsx')
                                                                   .to_dict('index').items())}

# Get port name
msc_delay_sheet = msc_delay_sheet.assign(
    pol_name=lambda x: x['Port of Loading'].apply(
        lambda y: msc_port_mapping.get(y)),
    pod_name=lambda x: x['Port of discharge'].apply(
        lambda y: msc_port_mapping.get(y))
)

In [11]:
session = requests.Session()

In [ ]:
msc_delay_sheet.drop_duplicates('Port of Loading')

In [ ]:
# Further filter by POL

Now I just need to figure out how to get the first cookie and use it in subsequent headers.

In [ ]:
def json_payload(port_of_loading):
    return {
        'mscCode': port_of_loading,
        'isCountry': False
    }

### MSCExtractor
Currently, there are 2 main Web APIs which we can use for data extraction. These are:
1. MSC Search Schedules API
    - countryID API
2. MSC Arrival-Departure API

The Search Schedules API is forward looking and is the preferred mode of extraction. We will extract information for all lines for this first. If the information extraction fails, we will than treat the line as having a vessel that has already left port. We then use the Arrival-Departure API to extract the information for these.

The countryID API is a sub-API used jointly with the Search Schedules API, which we will need to query before using the Search Schedules API because the parameters for the Search Schedules API uses numbers instead of the UNLOCODEs.

### MSC Search Schedules API

Need the PortOfLoadId and PortOfDischargeID for every port..

In [ ]:
url = "https://www.msc.com/api/schedules/search?WeeksOut=8&DirectRoutes=false&Date=2020-08-11&From=1034&To=221"
headers = {
  'Accept': 'application/json',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
  'Content-Type': 'application/json',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Dest': 'empty',
  'Referer': 'https://www.msc.com/search-schedules',
  'Accept-Language': 'en-GB,en;q=0.9',
  'Cookie': 'CMSPreferredCulture=en-GB; ASP.NET_SessionId=tht5lkut0asln2goiskoagfe; UrlReferrer=https://www.google.com/; CurrentContact=8b0b2fea-705b-4a4f-b8bf-bb1cd6c982bc; MSCAgencyId=115867; BIGipServerkentico.app~kentico_pool=439883018.20480.0000; _ga=GA1.2.1736073830.1597290148; _gid=GA1.2.1289141279.1597290148; _gcl_au=1.1.345060449.1597290148; __hstc=100935006.13bb76c8a78a8d0a203a993ffef3a3f6.1597290148282.1597290148282.1597290148282.1; hubspotutk=13bb76c8a78a8d0a203a993ffef3a3f6; __hssrc=1; _ym_uid=15972901491036911544; _ym_d=1597290149; _ym_isad=1; newsletter-signup-cookie=temp-hidden; _hjid=3e183004-f562-4048-8b60-daccdf9c187c; _hjUserAttributesHash=2c3b62a0e1cd48bdfd4d01b922060e19; _hjCachedUserAttributes={"attributes":{"mscAgencyId":"115867"},"userId":null}; OptanonAlertBoxClosed=2020-08-13T03:42:45.080Z; CMSCookieLevel=200; VisitorStatus=11062214903; TS0142aef9=0192b4b6225179b1baa3b4d270b71a4eee782a0192338173beabaa471f306c2a13fe854bf6a7ac08ac21924991864aa7728c54559023beabd273d82285d5f943202adb58da417d61813232e89b240828c090f890c6a74dc4adfec38513d13447be4b5b4404d69f964987b7917f731b858f0c9880a139994b98397c4aeb5bd60b0d0e38ec9e5f3c97b13fb184b4e068506e6086954f8a515f2b7239d2e5c1b9c70f61ca74f736355c58648a6036e9b5d06412389ac41221c5cb740df99c84dc2bfef4a530dbc5e2577c189212eebac723d9ee9f98030f4bc6ca7d824ab313ae5fdd1eaa9886; OptanonConsent=isIABGlobal=false&datestamp=Thu+Aug+13+2020+11%3A43%3A36+GMT%2B0800+(Singapore+Standard+Time)&version=5.9.0&landingPath=NotLandingPage&groups=1%3A1%2C2%3A1%2C3%3A1%2C4%3A1%2C0_53017%3A1%2C0_53020%3A1%2C0_53018%3A1%2C0_53019%3A1%2C101%3A1&AwaitingReconsent=false'
}

response = session.request("GET", url, headers=headers)
response.json()

In [ ]:
response.json()[0]['Sailings'][0]

In [ ]:
for i in range(len(response.json()[0]['Sailings'])):
    

### MSC countryID API

In [41]:
msc_delay_sheet.loc[msc_delay_sheet['Port of discharge'] == 'KRBNP']

,Plnt,Req. Delivery Date,Shipment,Term,Sold-to-Party Name,Ship-to-Pty,Sales Ord.,Delivery,Description,Product Type,Vessel,Voyage,ETD Date,Disport ETA,Gross Weight,Port of Loading,Port of discharge,Incoterms Part2,No. of Containers,Container Type,MetPro Status,Fwd Agent,Booking Ref.,Reason for rejection description,No. of bundles,Item Status Information,Incoterms Part1,Shipping Cond,pol_name,pod_name
35,2503,2020-07-30,30012636.0,4020,MERCURIA ENERGY TRADING SA,STEINWEGKR,15017923,802093760,IS 22KG AA170.9 CNTR 44 N,IS,HOPE ISLAND,FC031R,2020-08-02,2020-09-02,494.378,NZBLU,KRBNP,"BUSAN NEW PORT, KOREA",20,TEU,SHIPPED,,EBKG00702644,NaN,480.0,NaN,CIF,31,"BLUFF, New Zealand","BUSAN, Korea, Republic of"
36,2503,2020-07-30,30012636.0,4020,MERCURIA ENERGY TRADING SA,STEINWEGKR,15017924,802093761,IS 22KG AA170.9 CNTR 44 N,IS,HOPE ISLAND,FC031R,2020-08-02,2020-09-02,494.758,NZBLU,KRBNP,"BUSAN NEW PORT, KOREA",20,TEU,SHIPPED,,EBKG00702736,NaN,480.0,NaN,CIF,31,"BLUFF, New Zealand","BUSAN, Korea, Republic of"
37,2503,2020-07-30,30012636.0,4020,MERCURIA ENERGY TRADING SA,STEINWEGKR,15017925,802093762,IS 22KG AA170.9 CNTR 44 N,IS,HOPE ISLAND,FC031R,2020-08-02,2020-09-02,494.243,NZBLU,KRBNP,"BUSAN NEW PORT, KOREA",20,TEU,SHIPPED,,EBKG00702737,NaN,480.0,NaN,CIF,31,"BLUFF, New Zealand","BUSAN, Korea, Republic of"
38,2503,2020-07-30,30012636.0,4020,MERCURIA ENERGY TRADING SA,STEINWEGKR,15017926,802093763,IS 22KG AA170.9 CNTR 44 N,IS,HOPE ISLAND,FC031R,2020-08-02,2020-09-02,495.041,NZBLU,KRBNP,"BUSAN NEW PORT, KOREA",20,TEU,SHIPPED,,EBKG00702738,NaN,480.0,NaN,CIF,31,"BLUFF, New Zealand","BUSAN, Korea, Republic of"
61,2503,2020-08-05,30012750.0,4200,"HYUNDAI GLOVIS CO., LTD.",NOVELISUL,15018232,802094776,IS 23KG AA170.4 CNT 44 N,IS,MSC ADITI,FC032R,2020-08-05,2020-09-05,488.863,NZBLU,KRBNP,"BUSAN NEW PORT, KOREA",25,TEU,SHIPPED,,EBKG00753102,NaN,475.0,NaN,CIF,31,"BLUFF, New Zealand","BUSAN, Korea, Republic of"
62,2503,2020-08-10,30012750.0,4200,"HYUNDAI GLOVIS CO., LTD.",NOVELISUL,15018233,802094777,IS 23KG AA170.4 CNT 44 N,IS,MSC ADITI,FC032R,2020-08-05,2020-09-05,508.589,NZBLU,KRBNP,"BUSAN NEW PORT, KOREA",26,TEU,SHIPPED,,EBKG00753112,NaN,494.0,NaN,CIF,31,"BLUFF, New Zealand","BUSAN, Korea, Republic of"
64,2502,2020-09-05,30012753.0,4100,"HYUNDAI GLOVIS CO., LTD.",HDGLOVKR,15018390,802095009,BK 560 X 1350 P1100 5100 BS CNTR N,BK,CONTSHIP TEN,KI030R,2020-08-05,2020-09-04,105.750,AUBEL,KRBNP,ANY KOREAN PORT,5,TEU,SHIPPED,,EBKG00753341,NaN,10.0,ETA 05/09,CIF,31,"BELL BAY, Australia","BUSAN, Korea, Republic of"
65,2502,2020-09-05,30012753.0,4100,"HYUNDAI GLOVIS CO., LTD.",HDGLOVKR,15018391,802095010,BK 560 X 1520 P1100 4900 BS CNTR N,BK,CONTSHIP TEN,KI030R,2020-08-05,2020-09-04,91.288,AUBEL,KRBNP,ANY KOREAN PORT,4,TEU,SHIPPED,,EBKG00753352,NaN,8.0,ETA 05/09,CIF,31,"BELL BAY, Australia","BUSAN, Korea, Republic of"
66,2502,2020-08-30,30012753.0,4100,NOVELIS KOREA LIMITED,HDGLOVKR,15018432,802095115,BK 560 X 1720 GRADE.C SCRAP,BK,CONTSHIP TEN,KI030R,2020-08-05,2020-09-04,13.411,AUBEL,KRBNP,Port of Busan and/or Ulsan,1,TEU,SHIPPED,,EBKG00759124,NaN,1.0,NaN,CIF,31,"BELL BAY, Australia","BUSAN, Korea, Republic of"
67,2502,2020-08-05,30012753.0,4100,NOVELIS KOREA LIMITED,HDGLOVKR,15018433,802095112,BK 560 X 1520 GRADE.C SCRAP,BK,CONTSHIP TEN,KI030R,2020-08-05,2020-09-04,87.582,AUBEL,KRBNP,Port of Busan and/or Ulsan,4,TEU,SHIPPED,,EBKG00762173,NaN,7.0,NaN,CIF,31,"BELL BAY, Australia","BUSAN, Korea, Republic of"


In [54]:
def query_id(port):
    url = f"https://www.msc.com/api/schedules/autocomplete?q={port}"
    return session.request("GET", url)

In [ ]:
# Create the countryID query dataframe

In [47]:
msc_locations = list(msc_delay_sheet.pol_name.unique()) + list(msc_delay_sheet.pod_name.unique())

In [48]:
msc_locations

['BLUFF, New Zealand',
 'BRISBANE, Australia',
 'BELL BAY, Australia',
 'SYDNEY, Australia',
 'BUSAN, Korea, Republic of',
 'KRISTIANSAND, Norway',
 'ASHDOD, Israel',
 'ROTTERDAM, Netherlands',
 'Gongyi, CHINA',
 'PORT KLANG (PELABUHAN KLANG), Malaysia',
 'PASIR GUDANG, JOHOR, Malaysia',
 'KAOHSIUNG, Taiwain (China)',
 'AUCKLAND, New Zealand',
 'SHANGHAI, China',
 'JAKARTA, JAVA, Indonesia',
 'VUNG TAU, Viet Nam',
 'NAGOYA, Japan',
 'GWANGYANG, Korea, Republic of',
 'Zhenjiang, China',
 'CONSTANTA, Romania',
 'HOUSTON, United States',
 'YOKOHAMA, Japan',
 'OSAKA, Japan',
 'LAEM CHABANG, Thailand',
 'SURABAYA, Indonesia']

In [60]:
responses = {location: query_id(location) for location in msc_locations}

In [ ]:
def get_id(response):
    if len(response):
        return response.get('id')

In [59]:
[response.json() for response in responses]

[[{'id': 172,
   'name': 'BLUFF, NEW ZEALAND (NZBLU)',
   'isCountry': False,
   'countryId': 6578}],
 [{'id': 495,
   'name': 'BRISBANE, AUSTRALIA (AUBNE)',
   'isCountry': False,
   'countryId': 479}],
 [{'id': 1034,
   'name': 'BELL BAY, AUSTRALIA (AUBEL)',
   'isCountry': False,
   'countryId': 479}],
 [{'id': 489,
   'name': 'SYDNEY, AUSTRALIA (AUSYD)',
   'isCountry': False,
   'countryId': 479}],
 [{'id': 274,
   'name': 'BUSAN, KOREA, REPUBLIC OF (KRPUS)',
   'isCountry': False,
   'countryId': 4983}],
 [{'id': 34,
   'name': 'KRISTIANSAND, NORWAY (NOKRS)',
   'isCountry': False,
   'countryId': 6491}],
 [{'id': 318,
   'name': 'ASHDOD, ISRAEL (ILASH)',
   'isCountry': False,
   'countryId': 3945}],
 [{'id': 208,
   'name': 'ROTTERDAM, NETHERLANDS (NLRTM)',
   'isCountry': False,
   'countryId': 6325}],
 [],
 [{'id': 221,
   'name': 'PORT KLANG (PELABUHAN KLANG), MALAYSIA (MYPKG)',
   'isCountry': False,
   'countryId': 5465}],
 [{'id': 220,
   'name': 'PASIR GUDANG, JOHOR, MAL

In [19]:
response = query_id()

In [49]:
response.json()

[{'id': 172,
  'name': 'BLUFF, NEW ZEALAND (NZBLU)',
  'isCountry': False,
  'countryId': 6578}]

In [51]:
response.json()[0]['id']

172

### MSC Arrival-Departure API

In [ ]:
url = "https://www.msc.com/Site/WebServices/RouteFinder.svc/PortActivity"
payload = {
    'mscCode': 'NZBLU',
    'isCountry': False
}
headers = {
    'Accept': '*/*',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
    'Content-Type': 'application/json; charset=UTF-8',
    'Origin': 'https://www.msc.com',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.msc.com/arrivals-departures',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en;q=0.9',
    'Cookie': 'CMSPreferredCulture=en-GB; ASP.NET_SessionId=tht5lkut0asln2goiskoagfe; UrlReferrer=https://www.google.com/; CurrentContact=8b0b2fea-705b-4a4f-b8bf-bb1cd6c982bc; CMSLandingPageLoaded=true; MSCAgencyId=115867; BIGipServerkentico.app~kentico_pool=439883018.20480.0000; _ga=GA1.2.1736073830.1597290148; _gid=GA1.2.1289141279.1597290148; _gcl_au=1.1.345060449.1597290148; __hstc=100935006.13bb76c8a78a8d0a203a993ffef3a3f6.1597290148282.1597290148282.1597290148282.1; __hssrc=1; hubspotutk=13bb76c8a78a8d0a203a993ffef3a3f6; _ym_uid=15972901491036911544; _ym_d=1597290149; _ym_isad=1; _ym_visorc_65601397=w; newsletter-signup-cookie=temp-hidden; _hjid=3e183004-f562-4048-8b60-daccdf9c187c; _hjUserAttributesHash=2c3b62a0e1cd48bdfd4d01b922060e19; _hjCachedUserAttributes={`"attributes`":{`"mscAgencyId`":`"115867`"},`"userId`":null}; OptanonAlertBoxClosed=2020-08-13T03:42:45.080Z; CMSCookieLevel=200; VisitorStatus=11062214903; CMSUserPage={`"TimeStamp`":`"2020-08-13T05:43:28.7403678+02:00`",`"LastPageDocumentID`":2969,`"LastPageNodeID`":6383,`"Identifier`":`"93e5bd8e-a77a-49db-b00e-1e3712608ca2`"}; TS0142aef9=0192b4b6225179b1baa3b4d270b71a4eee782a0192338173beabaa471f306c2a13fe854bf6a7ac08ac21924991864aa7728c54559023beabd273d82285d5f943202adb58da417d61813232e89b240828c090f890c6a74dc4adfec38513d13447be4b5b4404d69f964987b7917f731b858f0c9880a139994b98397c4aeb5bd60b0d0e38ec9e5f3c97b13fb184b4e068506e6086954f8a515f2b7239d2e5c1b9c70f61ca74f736355c58648a6036e9b5d06412389ac41221c5cb740df99c84dc2bfef4a530dbc5e2577c189212eebac723d9ee9f98030f4bc6ca7d824ab313ae5fdd1eaa9886; OptanonConsent=isIABGlobal=false&datestamp=Thu+Aug+13+2020+11%3A43%3A36+GMT%2B0800+(Singapore+Standard+Time)&version=5.9.0&landingPath=NotLandingPage&groups=1%3A1%2C2%3A1%2C3%3A1%2C4%3A1%2C0_53017%3A1%2C0_53020%3A1%2C0_53018%3A1%2C0_53019%3A1%2C101%3A1&AwaitingReconsent=false; _gat=1; _gat_local=1; __hssc=100935006.3.1597290148283; _gali=ui-id-2'
}

In [ ]:
response = session.request("POST", url, headers=headers, json=payload)
response.json()

In [ ]:
print(session.cookies.get_dict())

### G2Extractor

In [69]:
class G2Extractor:
    def __init__(self, g2_file, xl, carrier_mapping):
        self.schedule = pd.read_excel(
            g2_file, skiprows=9, index_col='Unnamed: 0')
        self.delay_sheet = (xl.parse(pd.to_datetime(xl.sheet_names,
                                                    errors='coerce',
                                                    format='%d.%m.%Y').max().date().strftime('%d.%m.%Y'),
                                     parse_dates=True)
                            .query(f"`Fwd Agent` in {['G2OCEAN']}"))
        self.g2_port_map = {
            'AUPTJ': 'Portland',
            'AUNTL': 'Newcastle',
            'AUGLT': 'Gladstone',
            'NZTWI': 'Bluff',
            'TWKHH': 'Kaohsiung',
            'KRINC': 'Inchon',
            'KRPUS': 'Busan',
            'JPYOK': 'Yokohama',
            'JPNGO': 'Nagoya',
            'JPOSA': 'Osaka',
            'JPTOY': 'Toyama',
            'JPIHA': 'Niihama',
            'HKHKG': 'Hong Kong',
            'CNSHA': 'Shanghai'
        }

    def get_updated_etd(self, row):
        try:
            # column_index_etd is the column number that points to the ETD
            column_index_etd = np.argwhere(
                self.schedule.columns.str.contains(row['Vessel']))[0][0] + 1
        except IndexError:
            return np.nan
        return self.schedule.loc[self.schedule.index == self.g2_port_map.get(row['Port of Loading'])].iloc[:, column_index_etd][0]

    def get_updated_eta(self, row):
        try:
            # column_index_eta is the column number that points to the ETA
            column_index_eta = np.argwhere(
                self.schedule.columns.str.contains(row['Vessel']))[0][0]
        except IndexError:
            return np.nan
        return self.schedule.loc[self.schedule.index == self.g2_port_map.get(row['Port of discharge'])].iloc[:, column_index_eta][0]

    def extract(self):
        self.delay_sheet['updated_etd'] = self.delay_sheet.apply(
            self.get_updated_etd, axis=1)
        self.delay_sheet['updated_eta'] = self.delay_sheet.apply(
            self.get_updated_eta, axis=1)

### Final update

In [61]:
class DelayReport:
    def __init__(self):
        # Read configuration file
        with open("data/config.json", "r") as f:
            self.config = json.load(f)
            
        # Used to map carrier names to the ones BigSchedules uses and supports
        with open("data/carrier_mapping.json", "r") as f:
            self.carrier_mapping = json.load(f)
        
        # BigSchedules login
        with open("data/bigschedules_login.json", "r") as f:
            self.bs_login = json.load(f)
        
        # Prepare base information
        # UNLOCODE to port name mapping
        self.port_mapping = (
            pd.concat([pd.read_csv(p, usecols=[1, 2, 4, 5], engine='python', names=[
                      'country', 'port', 'name', 'subdiv']) for p in Path('data').glob("*UNLOCODE CodeListPart*")])
            .query('port == port')
            .assign(uncode=lambda x: x.country.str.cat(x.port),
                    full_name=lambda x: np.where(x.subdiv.notnull(), x.name.str.cat(x.subdiv, sep=", "), x.name))
            .drop_duplicates('uncode')
            .set_index('uncode')
            .to_dict('index'))
        
        # Read the vessel delay tracking file
        self.xl = pd.ExcelFile('Vessel Delay Tracking.xlsx')
        # today_date = datetime.now().strftime('%d.%m.%Y')
        # if today_date not in self.xl.sheet_names:
        #     raise Exception(
        #         f"The script cannot find today's date ({today_date}) in the Vessel Delay Tracking.xlsx file provided. Please check that the sheets are correctly named - the script will only operate on a sheet with today's date.")
        
    def run_bs(self):
        if self.config.get('run_bs'):
            bs_extractor = BSExtractor()
            bs_extractor.extract()
        
    def run_msc(self):
        if self.config.get('run_msc'):
            self.msc_extractor = MSCExtractor(self.xl, self.carrier_mapping)
            self.msc_extractor.extract()
    
    def run_g2(self):
        if self.config.get('run_g2'):
            self.g2_extractor = G2Extractor(self.config.get('g2_filename'), self.xl, self.carrier_mapping)
            self.g2_extractor.extract()
    
    def assemble(self):
        # Assemble the final dataframe to update
        main_delay_sheet = self.xl.parse()

        # Add new columns to the right side of the dataframe
        new_columns = ['updated_etd', 'updated_eta', 'No. of days delayed ETD', 'No. of days delayed ETA', 'Reason of Delay']
        main_delay_sheet[new_columns] = pd.DataFrame([[pd.NaT for i in range(4)] + [np.nan]])

        if self.config.get('run_bs'):
            main_delay_sheet.update(self.bs_extractor.delay_sheet)
        
        if self.config.get('run_msc'):
            main_delay_sheet.update(self.msc_extractor.delay_sheet)
        
        if self.config.get('run_g2'):
            main_delay_sheet.update(self.g2_extractor.delay_sheet)

        # Calculate the deltas
        main_delay_sheet['No. of days delayed ETD'] = (main_delay_sheet.updated_etd
                                                       - pd.to_datetime(main_delay_sheet['ETD Date'])).dt.days
        main_delay_sheet['No. of days delayed ETA'] = (main_delay_sheet.updated_eta
                                                       - pd.to_datetime(main_delay_sheet['Disport ETA'])).dt.days

        # Format the dates correctly via strftime
        date_columns = ['ETD Date', 'Disport ETA', 'updated_etd', 'updated_eta']
        for column in date_columns:
            main_delay_sheet[column] = main_delay_sheet[column].dt.strftime('%d/%m/%Y')
        self.main_delay_sheet = main_delay_sheet.copy()
    
    def output(self):
        # Output the excel file
        saved_file = f"Vessel Delay Tracking - {date.today().strftime('%d.%m.%Y')}.xlsx"
        self.main_delay_sheet.to_excel(saved_file)
        # os.startfile(saved_file)

In [70]:
# Delay report skeleton
delay_report = DelayReport()
delay_report.run_bs()
delay_report.run_msc()
delay_report.run_g2()
delay_report.assemble()